In [7]:
import numpy as np
import pandas as pd
import sys
import os
from pathlib import Path
if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
import glob
import tqdm
from sglm import utils, glm_fit
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet, Ridge, LinearRegression
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
# set path location to folder containing GLM_SIGNALS... and GLM_TABLE... .csv files
src = Path(r'E:\Dropbox (BOSTON UNIVERSITY)\2ABT\processed_photometry\9010\GLM_data')

def create_basis(file_dir):
    files = [Path(_).parts[-1] for _ in glob.glob(str(file_dir)) if 'SIGNALS' in _]

    basis = pd.DataFrame(files, columns=['signal_file'])
    basis['table_file'] = basis['signal_file'].str.replace('GLM_SIGNALS_', 'GLM_TABLE_')
    basis['figname'] = 'fig'
    basis['groupid'] = 'g'
    basis['mouseid'] = basis['signal_file'].str.replace('GLM_SIGNALS_', '').str.slice(0,8)
    basis['rename_dict'] = "{'Ch1': 'GCAMP'}"
    basis['signal_file_out'] = basis['signal_file'].str.replace('GLM_SIGNALS_', 'GLM_SIGNALS_INTERIM_')
    basis['table_file_out'] = basis['table_file'].str.replace('GLM_TABLE_', 'GLM_TABLE_INTERIM_')
    return basis

basis_df = create_basis(src / '*')
basis_df.to_csv(src / '_basis.csv', index=False)

In [9]:
num_sft = (-20, 20)

default_source_folder = Path(r'E:\Dropbox (BOSTON UNIVERSITY)\2ABT\processed_photometry\9010\GLM_data')
default_output_folder = Path(r'E:\Dropbox (BOSTON UNIVERSITY)\2ABT\processed_photometry\9010\GLM_data\interim')

output_json_name = Path('preproc_params.json')
basis_name = Path('_basis.csv')

y_col_lst_all = ['GCAMP', 'Ch1']

In [10]:
basis = pd.read_csv(str((default_source_folder / basis_name).resolve()))
basis.to_csv(str((default_output_folder / basis_name).resolve()))
basis = basis.sort_values('signal_file').reset_index(drop=True)
source_folder = default_source_folder

In [15]:
from tqdm.notebook import tqdm
import scipy.stats as stats
from collections import defaultdict
from sglm import gen_signal_df as gsd
from sglm import build_features as bf

basis = pd.read_csv(str((default_source_folder / basis_name).resolve()))
basis.to_csv(str((default_output_folder / basis_name).resolve()))
basis = basis.sort_values('signal_file').reset_index(drop=True)
source_folder = default_source_folder

interim_preproc_errors = defaultdict(list)

for inx, conversion in tqdm(basis.iterrows(), total=len(basis)):
#     print(conversion)
#     if 'source_folder' in conversion.index:
#         source_folder = Path(conversion['source_folder'])
#     else:
#         source_folder = default_source_folder
    
    figname = Path(conversion['figname'])
    mouse_ID = Path(conversion['mouseid'])
    groupid = Path(conversion['groupid'])
    signal_file = Path(conversion['signal_file'])
    table_file = Path(conversion['table_file'])
    rename_dict = eval(conversion['rename_dict'])
    signal_file_out = Path(conversion['signal_file_out'])
    table_file_out = Path(conversion['table_file_out'])
    
    
    signal_path = str((source_folder / signal_file).resolve())
    table_path = str((source_folder / table_file).resolve())
    
    out_folder = default_output_folder / figname / groupid
    out_folder.mkdir(parents=True, exist_ok=True)
    
    signal_path_out = default_output_folder / figname / groupid / signal_file_out
    table_path_out = default_output_folder / figname / groupid / table_file_out
    
    if signal_path_out.exists():
        raise OSError(f'Signal Path {signal_path_out} already exists!')
    if table_path_out.exists():
        raise OSError(f'Table Path {table_path_out} already exists!')
    
    signal_df = pd.read_csv(signal_path)
    table_df = pd.read_csv(table_path)
    
    brk = False
    # Check for multiple copies of the same sideIn
    for multiple_copy_check in ['photometryCenterInIndex', 'photometryCenterOutIndex',
                                'photometrySideInIndex', 'photometrySideOutIndex',]:
        eq = table_df[multiple_copy_check].dropna()
        eq = eq[eq != 0]
        if len(eq) != eq.nunique():
            interim_preproc_errors[(figname, groupid)].append({'in_file_errors': signal_file, 'out_file_errors': signal_file_out})

            print(len(eq))
            print(eq.nunique())
            display(eq)
            print(f'Error: Multiple {multiple_copy_check} with the same timestamp detected for {signal_path}. Continuing...')
            brk = True
            break
        elif (figname, groupid) not in interim_preproc_errors:
            interim_preproc_errors[(figname, groupid)] = []
    
    if brk:
        continue
    elif "GLM_SIGNALS_s stflp26_01252021" in str(signal_file.resolve()):
        print(f'Excluding {signal_file} and continuing...')
        interim_preproc_errors[(figname, groupid)].append({'in_file_errors': signal_file, 'out_file_errors': signal_file_out})
        continue
    
    
    print(signal_path)
    signal_df, table_df = gsd.generate_signal_df(signal_path,
                                            table_path,
                                            trial_bounds_before_center_in = num_sft[0],
                                            trial_bounds_after_side_out = num_sft[1],
                                            )
    
    assert np.all(table_df['choseLeft'] != table_df['choseRight'])
    
    signal_df = signal_df[signal_df['nTrial'] > 0].fillna(0)

    #adding SessionName 
    sn_string = basis['signal_file'][0].split('_')
    signal_df['SessionName'] = sn_string[2] + sn_string[3][0:-4]
    
    
    # Rename Columns
    signal_df = bf.rename_consistent_columns(signal_df)
    signal_df = signal_df.rename(rename_dict, axis=1)

    for y_col in y_col_lst_all:
        if y_col not in signal_df.columns:
            signal_df[y_col] = np.nan
            continue
    
    ## Set Full Trial Reward Flags
    signal_df['r_trial'] = (signal_df.groupby('TrialNumber')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
    signal_df['nr_trial'] = (signal_df.groupby('TrialNumber')['photometrySideInIndexnr'].transform(np.sum) > 0) * 1.0
    
    signal_df['nr_trial'] = (signal_df['nr_trial'] - signal_df['r_trial'] > 0) * 1.0
    
#     signal_df['rnr_err_trial'] = (signal_df['r_trial'] + signal_df['nr_trial'] != 1.0) * 1.0

    ## Define Side Rewarded / Unrewarded Flags
    signal_df = bf.set_port_entry_exit_rewarded_unrewarded_indicators(signal_df)
    
#     display(list(signal_df.columns))
    
    ## Define Side Agnostic Events
    signal_df = bf.define_side_agnostic_events(signal_df, agnostic_definitions={'spn':['rpn','lpn'],
                                                          'spx':['rpx','lpx'],
                                                          'spnr':['rpnr','lpnr'],
                                                          'spnnr':['rpnnr','lpnnr'],
                                                          'spxr':['rpxr','lpxr'],
                                                          'spxnr':['rpxnr','lpxnr'],
#                                                           'sl':['rl','ll'],
                                                         })
    
    #print('Percent of Data in ITI:', (signal_df['nTrial'] == signal_df['nEndTrial']).mean())

    signal_df['spnrOff'] = ((signal_df['spnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    signal_df['spxrOff'] = ((signal_df['spxr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)
    spnnrOff_a = ((signal_df['spnnr'] == 1)&(signal_df['photometrySideInIndex'] != 1)).astype(int)
    spxnrOff_a = ((signal_df['spxnr'] == 1)&(signal_df['photometrySideOutIndex'] != 1)).astype(int)

    # If we have something listed as a rewarded "off" side entry labeled in the table as a side exit... it means it was a fast "out-in".
    # The latter "in" should be considered an unrewarded side port "off" entry.
    dualism_exen = ((signal_df['spnrOff'] == 1)&(signal_df['photometrySideOutIndex'] == 1)).astype(int)

    # Unrewarded side port entries should be the combination of those simply identified by checking spnnr & the table labels +
    # the dualism defined immediately prior. Then those dualism examples should be remoed from the "off" rewarded entries.
    signal_df['spnnrOff'] = spnnrOff_a + dualism_exen
    signal_df['spnrOff'] = signal_df['spnrOff'] - dualism_exen
    signal_df['spxnrOff'] = spxnrOff_a
    
#     signal_df['slOff'] = signal_df['sl'] * signal_df['nr_trial']
#     signal_df['slOn'] = signal_df['sl'] - signal_df['slOff']
    
    signal_df['cpnOff'] = ((signal_df['cpn'] == 1)&(signal_df['photometryCenterInIndex'] != 1)).astype(int)
    signal_df['cpxOff'] = ((signal_df['cpx'] == 1)&(signal_df['photometryCenterOutIndex'] != 1)).astype(int)
    
    
    #for resp in ['GCAMP']:
#     for resp in y_col_lst_all:
    #    print(f'{resp} Stats:')
    #    a = np.round(signal_df.assign(wi_trial_keep=signal_df['wi_trial_keep'].astype(str)).groupby('wi_trial_keep')[resp].agg([np.mean, np.var, stats.skew, lambda a: stats.kurtosis(a,fisher=False)]),3)
    #    a = pd.concat([a,np.round(signal_df.assign(one='All').groupby('one')[resp].agg([np.mean, np.var, stats.skew, lambda a: stats.kurtosis(a,fisher=False)]),3)],axis=0)
    #    display(a)
    
    if signal_path_out:
        signal_df.to_csv(signal_path_out, index_label='Timestamp')
    if table_path_out:
        table_df.to_csv(table_path_out, index_label='Timestamp')
    
    assert np.max(signal_df['photometrySideInIndexr'] + signal_df['photometrySideInIndexnr']) == 1.0

#signal_df['Timestamp'] = signal_df['index']
    
#     display(signal_df[['GCAMP']].agg([np.mean, np.std, scipy.stats.skew, scipy.stats.kurtosis]))

for figname, groupid in tqdm(interim_preproc_errors, total=len(interim_preproc_errors)):
    interim_preproc_errors_path = default_output_folder / figname / groupid / 'interim_preproc_errors.csv'

    if len(interim_preproc_errors[(figname, groupid)]) != 0:
        interim_preproc_errors_df = pd.DataFrame(interim_preproc_errors[(figname, groupid)])
    else:
        interim_preproc_errors_df = pd.DataFrame(columns=['in_file_errors', 'out_file_errors'])

    interim_preproc_errors_df.to_csv(str(interim_preproc_errors_path.resolve()))

  0%|          | 0/49 [00:00<?, ?it/s]

E:\Dropbox (BOSTON UNIVERSITY)\2ABT\processed_photometry\9010\GLM_data\GLM_SIGNALS_sstflp15_05302019.csv
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1


C:\Users\mwall\Documents\GitHub\sabatini-glm-workflow\sglm\gen_signal_df.py:105: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  check['val_a'] = df_t['label'].str.slice(0, 1).replace('a', 0).replace('A', 1) + df_t['label'].str.slice(1, 2).replace('b', 0).replace('B', 2).replace('a', 4).replace('A', 6)
C:\Users\mwall\AppData\Local\Temp\ipykernel_13676\2922615450.py:100: FutureWarning: The provided callable <function sum at 0x0000020429FDC280> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  signal_df['r_trial'] = (signal_df.groupby('TrialNumber')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
C:\Users\mwall\AppData\Local\Temp\ipyke

E:\Dropbox (BOSTON UNIVERSITY)\2ABT\processed_photometry\9010\GLM_data\GLM_SIGNALS_sstflp15_06072019.csv
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1


C:\Users\mwall\Documents\GitHub\sabatini-glm-workflow\sglm\gen_signal_df.py:105: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  check['val_a'] = df_t['label'].str.slice(0, 1).replace('a', 0).replace('A', 1) + df_t['label'].str.slice(1, 2).replace('b', 0).replace('B', 2).replace('a', 4).replace('A', 6)
C:\Users\mwall\AppData\Local\Temp\ipykernel_13676\2922615450.py:100: FutureWarning: The provided callable <function sum at 0x0000020429FDC280> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  signal_df['r_trial'] = (signal_df.groupby('TrialNumber')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
C:\Users\mwall\AppData\Local\Temp\ipyke

...

E:\Dropbox (BOSTON UNIVERSITY)\2ABT\processed_photometry\9010\GLM_data\GLM_SIGNALS_sstflp26_02182021.csv
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1


C:\Users\mwall\Documents\GitHub\sabatini-glm-workflow\sglm\gen_signal_df.py:105: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  check['val_a'] = df_t['label'].str.slice(0, 1).replace('a', 0).replace('A', 1) + df_t['label'].str.slice(1, 2).replace('b', 0).replace('B', 2).replace('a', 4).replace('A', 6)
C:\Users\mwall\AppData\Local\Temp\ipykernel_13676\2922615450.py:100: FutureWarning: The provided callable <function sum at 0x0000020429FDC280> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  signal_df['r_trial'] = (signal_df.groupby('TrialNumber')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
C:\Users\mwall\AppData\Local\Temp\ipyke

E:\Dropbox (BOSTON UNIVERSITY)\2ABT\processed_photometry\9010\GLM_data\GLM_SIGNALS_sstflp26_02232021.csv
# of iterations 0 — Final max amount of duplicated Center Out Indices: 1


C:\Users\mwall\Documents\GitHub\sabatini-glm-workflow\sglm\gen_signal_df.py:105: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  check['val_a'] = df_t['label'].str.slice(0, 1).replace('a', 0).replace('A', 1) + df_t['label'].str.slice(1, 2).replace('b', 0).replace('B', 2).replace('a', 4).replace('A', 6)
C:\Users\mwall\AppData\Local\Temp\ipykernel_13676\2922615450.py:100: FutureWarning: The provided callable <function sum at 0x0000020429FDC280> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  signal_df['r_trial'] = (signal_df.groupby('TrialNumber')['photometrySideInIndexr'].transform(np.sum) > 0) * 1.0
C:\Users\mwall\AppData\Local\Temp\ipyke

  0%|          | 0/1 [00:00<?, ?it/s]

In [9]:
%pdb off

Automatic pdb calling has been turned OFF
